In [1]:
import pandas as pd
import xgboost as xgb
import lightgbm as lgb

In [2]:
df = pd.read_csv('log.csv', index_col=0)

for col in df.columns:
    if not col == "response":
        for i in range(1,3):
            df[col+" lag_{}".format(i)] = df[col].shift(i)
            

df["response_rolling_2"] = df["response"].rolling(2).mean()
df = df.dropna()
rolling = df["response_rolling_2"].copy()
df = df.iloc[:,:-1].astype(int)

In [3]:
rolling

2     -1.0
3      0.5
4      0.5
5     -1.0
6      0.5
7      0.5
8     -1.0
9      0.5
10     0.5
11    -1.0
12     0.5
13     0.5
14     0.5
15     0.5
16     0.5
17     0.5
18    -1.0
19    -1.0
20    -1.0
21     0.5
22     0.5
23    -1.0
24    -1.0
25    -1.0
26    -1.0
27     0.5
28     0.5
29    -1.0
30    -1.0
31     0.5
      ... 
970    0.5
971    2.0
972    0.5
973   -1.0
974   -1.0
975    0.5
976    0.5
977   -1.0
978    0.5
979    0.5
980    0.5
981    2.0
982    2.0
983    2.0
984    0.5
985   -1.0
986   -1.0
987   -1.0
988    0.5
989    0.5
990    0.5
991    2.0
992   -4.0
993   -4.0
994    2.0
995    0.5
996   -1.0
997   -1.0
998   -1.0
999   -1.0
Name: response_rolling_2, Length: 998, dtype: float64

In [4]:
train_y = rolling
train_X = df[df.columns[1:]]

In [5]:
train_X.tail(15)

,touch_sensor,turn right,turn left,forward,backward,check,touch_sensor lag_1,touch_sensor lag_2,turn right lag_1,turn right lag_2,turn left lag_1,turn left lag_2,forward lag_1,forward lag_2,backward lag_1,backward lag_2,check lag_1,check lag_2
985,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0
986,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
987,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
988,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0
989,-1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0
990,0,0,0,1,0,0,-1,0,0,0,0,0,0,1,0,0,1,0
991,0,0,0,1,0,0,0,-1,0,0,0,0,1,0,0,0,0,1
992,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0
993,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0
994,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [6]:
newrow = pd.DataFrame(columns = train_X.columns)
newrow.loc[0]= [0 for _ in range(len(train_X.columns))]

item = list(train_X.columns)[:6]
for col in item:
    for i in range(1,3):
        newrow.loc[0][col+" lag_{}".format(i)] = train_X.loc[len(train_X)-i+2][col]

In [7]:
newrow = newrow.astype(int)

In [19]:
model = xgb.XGBRegressor(objective='reg:squarederror')
model.fit(train_X, train_y)

c:\users\lanth\appdata\local\programs\python\python37\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

In [12]:
newrow.loc[0]["forward lag_2"] = 1
newrow

,touch_sensor,turn right,turn left,forward,backward,check,touch_sensor lag_1,touch_sensor lag_2,turn right lag_1,turn right lag_2,turn left lag_1,turn left lag_2,forward lag_1,forward lag_2,backward lag_1,backward lag_2,check lag_1,check lag_2
0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0


In [22]:
pred = {}
for feat in ["turn right",
             "turn left",
             "forward",
             "backward"]:
    tmp = newrow.copy()
    tmp.loc[0][feat] = 1
    pred[feat] = model.predict(tmp)[0]

tmp = newrow.copy()
tmp.loc[0]["check"] = 1
tmp.loc[0]["touch_sensor"] = -1

pred["check"] = model.predict(tmp)[0]

In [23]:
for p in pred:
    print(p, pred[p])

turn right -0.51022446
turn left -0.64225566
forward -0.42899364
backward -0.3543929
check -0.5286534


In [24]:
model.predict(newrow)[0]

-0.3543929